In [16]:
from google.colab import drive
drive.mount('/content/drive')
project_path = '/content/drive/My Drive/Spam-Classification/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import ModelCheckpoint
from collections import Counter
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from nltk.stem import PorterStemmer 
from nltk import tokenize
from tensorflow.keras import backend as K
import nltk
import gensim.models.keyedvectors as word2vec #need to use due to depreceated model
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
import pandas as pd

[nltk_data] Downloading package punkt to /home/phat-ngu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-11-11 08:07:44--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.96.245
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.96.245|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [4]:
# Load data as a list of pairs of sentences and labels (0 for non-spam, 1 for spam)
data = []
with open('SMSSpamCollection') as f:
    for line in f:
        firstword = (line.split()[0])
        label = None
        if firstword == 'ham':
            label = 0
        else: 
            label = 1
        text = line.replace(firstword,'').strip()
        data.append([text,label])
# transform data to numpy array
data = pd.DataFrame(np.array(data),columns=['text','label'])

In [5]:
mdcp = ModelCheckpoint(filepath=project_path + 'weights.{epoch:02d}-{val_f1_m:.2f}',monitor='val_f1_m')

NameError: name 'project_path' is not defined

In [6]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
    
#Dividing the dataset into features and lables
X = data['text']
Y = data['label']


In [7]:
max_words = 3500
max_len = 50
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [8]:
X_train,X_test,Y_train,Y_test=train_test_split(sequences_matrix,Y,test_size=0.15,random_state=4)

In [9]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.25)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy',recall_m,precision_m,f1_m])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 50)            175000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257   

In [109]:
model.fit(X_train,Y_train,batch_size=64,epochs=10,
          validation_split=0.2,callbacks=[mdcp])

Train on 3789 samples, validate on 948 samples
Epoch 1/10
3789/3789 [==============================] - 15s 4ms/step - loss: 0.2233 - acc: 0.9171 - recall_m: 0.4176 - precision_m: 0.5778 - f1_m: 0.4671 - val_loss: 0.0713 - val_acc: 0.9852 - val_recall_m: 0.9007 - val_precision_m: 0.9904 - val_f1_m: 0.9409
Epoch 2/10
3789/3789 [==============================] - 7s 2ms/step - loss: 0.0449 - acc: 0.9860 - recall_m: 0.9176 - precision_m: 0.9732 - f1_m: 0.9413 - val_loss: 0.0393 - val_acc: 0.9873 - val_recall_m: 0.9266 - val_precision_m: 0.9735 - val_f1_m: 0.9462
Epoch 3/10
3789/3789 [==============================] - 7s 2ms/step - loss: 0.0275 - acc: 0.9918 - recall_m: 0.9570 - precision_m: 0.9834 - f1_m: 0.9677 - val_loss: 0.0356 - val_acc: 0.9884 - val_recall_m: 0.9363 - val_precision_m: 0.9735 - val_f1_m: 0.9513
Epoch 4/10
3789/3789 [==============================] - 7s 2ms/step - loss: 0.0173 - acc: 0.9952 - recall_m: 0.9766 - precision_m: 0.9827 - f1_m: 0.9784 - val_loss: 0.0368 - val_

In [111]:
path_model = "/content/drive/My Drive/Spam-Classification/weights.08-0.95"
model.load_weights(path_model)
model.evaluate(X_test,Y_test,verbose=1)

837/837 [==============================] - 1s 1ms/step


[0.12895864789407574,
 0.9904420549581839,
 0.9422616696557121,
 0.9876543202280571,
 0.9614949197894499]

In [10]:
X_train,X_test,Y_train,Y_test=train_test_split(sequences_matrix,Y,test_size=0.15,random_state=8)
path_model = "weights-9529-test-9615-best.h5"
model.load_weights(path_model)
model.evaluate(X_test,Y_test,verbose=1)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'pandas.core.series.Series'>

In [132]:
path_model = "/content/drive/My Drive/Spam-Classification/weights-9529-test-9614-best"
records = []
for i in range(5):
  X_train,X_test,Y_train,Y_test=train_test_split(sequences_matrix,Y,test_size=0.15,random_state=i)
  model.load_weights(path_model)
  _,accuracy,recall,precision,f1=model.evaluate(X_test,Y_test)
  records.append([accuracy,precision,recall,f1])
records_df = pd.DataFrame(records,columns=['Accuracy','Recall','Precision','F1-Score'])

837/837 [==============================] - 1s 1ms/step


In [133]:
records_df

,Accuracy,Recall,Precision,F1-Score
0,0.996416,0.955794,0.939409,0.946482
1,0.996416,0.970450,0.994026,0.980841
2,0.995221,0.986380,0.973636,0.978579
3,0.995221,0.952210,0.934005,0.940258
4,0.990442,0.987654,0.942262,0.961495


In [129]:
records_df


,Accuracy,Recall,Precision,F1-Score
0,0.992832,0.949423,0.930472,0.937131
1,0.994026,0.925846,0.946237,0.932977
2,0.989247,0.987654,0.921598,0.947080
3,0.992832,0.956307,0.920199,0.935488
4,0.988053,0.981282,0.932704,0.952558


In [131]:
records_df


,Accuracy,Recall,Precision,F1-Score
0,0.997611,0.955794,0.944871,0.949913
1,0.995221,0.962804,0.994026,0.976593
2,0.995221,0.986380,0.973636,0.978579
3,0.995221,0.952210,0.934005,0.940258
4,0.989247,0.981282,0.942262,0.958019
